In [ ]:
!pip install paddlepaddle-gpu
!pip install paddleocr


In [ ]:
# !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
# !sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb


In [ ]:
# !git clone https://github.com/PaddlePaddle/PaddleOCR



In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
from IPython import display

OpenCV bindings requires "numpy" package.
Install it via command:
    pip install numpy


ImportError: cannot import name 'fastCopyAndTranspose' from 'numpy.core._multiarray_umath' (/usr/local/lib/python3.10/dist-packages/numpy/core/_multiarray_umath.py)

In [ ]:
! pip install -U spacy -q

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import spacy
from spacy.tokens import DocBin

In [ ]:
import re
import cv2
import pandas as pd
import difflib as diff
from tqdm import tqdm
import pandas as pd


In [ ]:
# UPLOAD FILES and UNZIP IT
import zipfile
import os

# Specify the path to the zip file
zip_file_path = '/content/project_files.zip'

# Specify the directory to extract the contents
extract_dir = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Folder extracted successfully.")


In [ ]:
inv_df = pd.DataFrame(columns=[])

In [ ]:
image_file = '/content/Images/IMG_0959.jpg'
img = cv2.imread(image_file)

thresh, im_bw = cv2.threshold(img, 110, 200, cv2.THRESH_BINARY)
#inverted_image = cv2.bitwise_not(img)
# ocr = PaddleOCR(lang='en')
ocr = PaddleOCR(use_angle_cls=True)

result = ocr.ocr(img, cls=True)

myresult = ""
for idx in range(len(result)):
    res = result[idx]
    for line in res:
      # text.append(line[1][0])
      myresult +=  f" {line[1][0]}"

In [ ]:
print(myresult)

In [ ]:
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
nlp_ner = spacy.load("/content/Model/model-best")

NameError: name 'spacy' is not defined

In [ ]:
# Used Regex

batch_number = re.search(r'(BatchNo,|BatchNo|Batch No.:|Batch No\.|BatchNo\.|Batch|B\.No\.|B\.NO\.|B\.No|BNo.|B.N|N0:|No.|.NO.)\s*(\w+)', myresult)

if batch_number:
    print("Batch number:", batch_number.group(2))
    bno = batch_number.group(2)
else:
    doc = nlp_ner(myresult)
    spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

    for ent in doc.ents:
      if ent.label_ == 'BATCH':
        bno = ent.text
        print(bno)

In [ ]:
bno

In [ ]:
import pandas as pd
from difflib import SequenceMatcher

df = pd.read_csv('/content/Dataset/Medicine_Data.csv')

text_column = df['Batch']

similarity_ratios = []
for i in text_column:
    ratio = SequenceMatcher(None, i, bno).ratio()
    similarity_ratios.append(ratio)

# Find the maximum similarity ratio
max_similarity = max(similarity_ratios)
df['Ratio']=similarity_ratios
# Print the maximum similarity ratio
print("Maximum similarity ratio:", max_similarity)

# saving the dataframe
df.to_csv('/content/Dataset/Similarity_Ratio.csv')

In [ ]:
df = pd.read_csv('/content/Dataset/Similarity_Ratio.csv')
# create a new empty dataframe

max_similarity = max(df['Ratio'])
for index, row in df.iterrows():
    if row['Ratio'] == max_similarity and max_similarity >0:
        #print(row['Company'],row['Name'],row['Batch'],row['MRP'],row['Mfg_dt'],row['Exp_dt'])
        print(row[1:7],"\n")
        Product=row['Name']
        BatchNo=row['Batch']
        Expiry_Date=row['Exp_dt']
        Price=row['MRP']
        inv_df = inv_df.append({'Product': Product,
                           'Batch': BatchNo,
                           'Expiry': Expiry_Date,
                           'Price': Price}, ignore_index=True)

In [ ]:
inv_df